## Εξίσωση διάδοσης θερμότητας

## Implicit μέθοδος (Crank - Nicolson)
________

[John Crank](https://en.wikipedia.org/wiki/John_Crank) & [Phyllis Nicolson](https://en.wikipedia.org/wiki/Phyllis_Nicolson)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd


plt.style.use("default")
plt.rcParams["figure.figsize"] = [5, 2.5]  # [width_inches, height_inches]
plt.rcParams["animation.html"] = "jshtml"

### Εφαρμογή: Παράδειγμα 2.2 του βιβλίου G.D. Smith

(σελ. 21 του βιβλίου, σελ. 19 του pdf)

Eξίσωση θερμότητας:

$$
\begin{equation*}
\frac{\partial u}{\partial t} = \frac{\partial^2 u}{\partial x^2}
\end{equation*}
$$

Αρχική συνθήκη:

- $u = 2x, \;$ για $\; 0 \leq x < 1/2, \quad t=0$

- $u = 2(1-x), \;$ για $\; 1/2 \leq x \leq 1, \quad t=0$

Οριακές συνθήκες:

- $u = 0,  \;$ για $\; x=0$

- $u = 0,  \;$ για $\; x=1$

Διακριτοποίηση αξόνων:

- Χωρικό βήμα: $\;  δx = h = 1/10$

- Χρονικό βήμα: $\; δt = k = 1/100$

- $r = \frac{k}{h^2} = 1$


### Implicit μέθοδος επίλυσης (Crank-Nicolson)

- Forward-difference ως προς το $t$

- Μέση τιμή της centered-difference ως προς το $x$ για τις \
χρονικές στιγμές: (α) $n$ και (β) $n+1$

$$
\begin{align*}
& \quad \; \frac{\partial u}{\partial t} = \frac{\partial^2 u}{\partial x^2} \\[15pt]
& \Rightarrow \frac {u^{n+1}_i - u^n_i}{k} = \frac{1}{2}(\frac{u^{n+1}_{i-1} - 2u^{n+1}_i + u^{n+1}_{i+1}}{h^2} + \frac{u^n_{i-1} - 2u^n_i + u^n_{i+1}}{h^2})\\[15pt]
& \Rightarrow 2u^{n+1}_i - 2u^n_i = r(u^{n+1}_{i-1} - 2u^{n+1}_i + u^{n+1}_{i+1} + u^n_{i-1} - 2u^n_i + u^n_{i+1})\\[15pt]
& \Rightarrow -ru^{n+1}_{i-1} + (2 + 2r)u^{n+1}_{i} - ru^{n+1}_{i+1} = ru^n_{i-1} + (2-2r)u^n_i + ru^n_{i+1} \\
\end{align*}
$$

- Διακριτοποίηση χ-άξονα

In [ ]:
x0 = 0
xN = 1
h = 1 / 10
Nx = int((xN - x0) / h + 1)
x = np.linspace(start=x0, stop=xN, num=Nx, endpoint=True, retstep=False)
x


- Διακριτοποίηση t-άξονα

In [ ]:
t0 = 0
k = 1 / 100
Nt = 120  # πλήθος χρονικών στιγμών
tN_plus_k = Nt * k
t = np.arange(t0, tN_plus_k, k)

- Δημιουργία πλέγματος

In [ ]:
xx, tt = np.meshgrid(x, t)

- Δημιουργία κενού πίνακα για την αριθμητική λύση

In [ ]:
u = np.full((Nt, Nx), np.nan)

- Αρχική συνθήκη

$\quad u = 2x, \;$ για $\; 0 \leq x < 1/2$

$\quad u = 2(1-x), \;$ για $\; 1/2 \leq x \leq 1$

In [ ]:
u[0, x < 0.5] = 2 * x[x < 0.5]
u[0, x >= 0.5] = 2 * (1 - x[x >= 0.5])

- Οριακές συνθήκες

In [ ]:
u[:, 0] = 0
u[:, -1] = 0

- Συντελεστής $r$

In [ ]:
r = k / (h**2)
r

### Αριθμητική λύση Implicit

$-ru^{n+1}_{i-1} + (2 + 2r)u^{n+1}_{i} - ru^{n+1}_{i+1} = ru^n_{i-1} + (2-2r)u^n_i + ru^n_{i+1}$


Οι εξισώσεις ανά σημείο είναι οι εξής:

$  -ru^{n+1}_{0} + (2 + 2r)u^{n+1}_{1} - ru^{n+1}_{2} = ru^n_{0} + (2-2r)u^n_1 + ru^n_{2}$

$  -ru^{n+1}_{1} + (2 + 2r)u^{n+1}_{2} - ru^{n+1}_{3} = ru^n_{1} + (2-2r)u^n_2 + ru^n_{3}$

$ \dots $

$  -ru^{n+1}_{8} + (2 + 2r)u^{n+1}_{9} - ru^{n+1}_{10} = ru^n_{8} + (2-2r)u^n_9 + ru^n_{10}$

&nbsp;

Σε μορφή πινάκων:

$A\vec{u}^{n+1}_i = B\vec{u}^{n}_i + \vec{a}^n + \vec{b}^n$

&nbsp;

Διαστάσεις πινάκων:

$\vec{u}^{n+1}_i: \;(N_x - 2)$

$A: \;(N_x - 2) \times (N_x - 2)$

$B: \;(N_x - 2) \times (N_x - 2)$

$\vec{u}^{n}_i: \;(N_x - 2) $

$\vec{a}^n: \;(N_x - 2)$

$\vec{b}^n: \;(N_x - 2)$

Τα $\; \vec{a}^n, \; \vec{b}^n$ μπορούν να αγνοηθούν στο παρόν πρόβλημα γιατί είναι \
ίσα με μηδέν για όλες τις χρονικές στιγμές.

$A\vec{u}^{n+1}_i = B\vec{u}^{n}_i$

Παρατήρηση:

Είναι προτιμητέο να μην μετασχηματίσουμε την εξίσωση στην παρακάτω μορφή, \
[αποφεύγοντας επομένως να βρούμε τον αντίστροφο του πίνακα](https://gregorygundersen.com/blog/2020/12/09/matrix-inversion/) καθώς είναι \
πιο αργή και λιγότερο ακριβής μέθοδος.

$\vec{u}^{n+1}_i = A^{-1}B\vec{u}^{n}_i$

### Εφαρμογή Implicit μεθόδου

Δημιουργήστε μηδενικούς πίνακες για τα $Α$ και $B$:

In [ ]:
A = np.zeros((Nx-2, Nx-2))
B = np.zeros((Nx-2, Nx-2))


Δώστε τις κατάλληλες τιμές στο $Α$:

- Διαγώνιος: $2+2r$

- Μετατοπισμένες διαγώνιοι: $-r$

In [ ]:
np.fill_diagonal(A, (2 + (2 * r)))  # in-place συνάρτηση

offset_diagonal_values = np.repeat(-r, A.shape[0] - 1)

A = A + np.diag(offset_diagonal_values, 1) + np.diag(offset_diagonal_values, -1)

Δώστε τις κατάλληλες τιμές στο $B$:

- Διαγώνιος: $2-2r$

- Μετατοπισμένες διαγώνιοι: $r$

In [ ]:
np.fill_diagonal(B, (2 - (2 * r)))  # in-place συνάρτηση

offset_diagonal_values = np.repeat(r, B.shape[0] - 1)

B = B + np.diag(offset_diagonal_values, 1) + np.diag(offset_diagonal_values, -1)

$A\vec{u}^{n+1}_i = B\vec{u}^{n}_i$

Βρείτε τη λύση της μερικής διαφορικής εξίσωσης επιλύοντας σε κάθε χρονική \
στιγμή το παραπάνω σύστημα εξισώσεων. 

Χρησιμοποιήστε τη συνάρτηση `np.linalg.solve`

In [ ]:
for n in range(Nt - 1):
    u[n + 1, 1:-1] = np.linalg.solve(A, B.dot(u[n, 1:-1]))


### Αναλυτική λύση της μερικής διαφορικής εξίσωσης

$$
\begin{equation*}
U = \frac{8}{\pi^2} \sum_{n=1}^{\infty}\frac{1}{n^2}\sin(\frac{1}{2}n\pi)\sin(n\pi x)e^{-n^2\pi^2t}
\end{equation*}
$$

In [ ]:
nterms = 100
terms = []
for n in range(1, nterms + 1):
    terms.append(
        (1/n**2) *
        np.sin(1/2 * n * np.pi) *
        np.sin(n * np.pi * xx) *
        np.exp(-n**2 * np.pi**2 * tt)
    )
U = (8/np.pi**2) * np.sum(terms, axis=0)

### Έλεγχος αποτελεσμάτων

In [ ]:
df_u = pd.DataFrame.from_records(
    np.round(u, 4),
    columns=np.round(x, 1),
    index=t,
)
df_u.head(6)

In [ ]:
df_U = pd.DataFrame.from_records(
    np.round(U, 4),
    columns=np.round(x, 1),
    index=t,
)

Table 2.9 του βιβλίου

Κρατήστε τις κατάλληλες χρονικές στιγμές και το κατάλληλο σημείο της ράβδου \
από τα `df_u` και τα `df_U` έτσι ώστε να συγκρίνετε τα αποτελέσματά σας με αυτά \
του πίνακα 2.9 του βιβλίου.

In [ ]:
times = [0.01, 0.02, 0.10]
position = 0.5
df_u = df_u.loc[df_u.index.isin(times)]
df_u[[position]]

In [ ]:
df_U = df_U.loc[df_U.index.isin(times)]
df_U[[position]]


### Animation με την αριθμητική και την αναλυτική λύση

In [ ]:
fig, ax = plt.subplots()
plt.close()


def animate(i):
    ax.clear()
    ax.plot(x, U[i], color="orange")
    ax.scatter(x, u[i], color="dimgrey")
    ax.set_ylim([-0.05, 1 + 0.05])
    ax.set_xlabel("x", fontsize=12)
    ax.set_ylabel("Temperature", fontsize=12)
    ax.set_title(f"Time: {t[i]:.2f} seconds", fontweight="bold", loc="center")

ani = FuncAnimation(
    fig=fig,
    func=animate,
    frames=Nt,
    interval=50,
    repeat=False,
)
plt.close()

In [ ]:
ani

Διπλασιάστε ή τριπλασιάστε το χρονικό βήμα και παρατηρήστε ότι η implicit \
μέθοδος παραμένει ευσταθής ακολουθώντας την αριθμητική λύση.

"_Although the Crank-Nicolson method is stable for all positive values of r \
in the sense that the solution and all errors eventually tend to zero as n \
tends to infinity, it will be shown that large values for r, such as 40, \
can introduce unwanted finite oscillations into the numerical solution_."

Εξετάστε αν δημιουργούνται στο παρόν πρόβλημα ταλαντώσεις δίνοντας μεγάλο \
χρονικό βήμα ($k = 0.4$)

(αναπροσαρμόστε την παράμετρο `interval` του `FuncAnimation` σε τιμή 200)